# 컨텐츠 기반 모델

## Word2Vec

```
✅ 통계기반의 방법 단점
- 대규모 말뭉치를 다룰 때 메모리상의 문제가 발생 
    - 높은 차원을 가짐 , 매우 sparse 한 형태의 데이터임 
    - 예) 100만개의 문서를 다루는 경우 : 100만개의 문서에 등장한 모든 단어를 추출해야하고 
      이때 단어의 수는 1문서당 새로운 단어가 10개면, 1000만개정도의 말뭉치가 형성됨.
      즉, 100만 x 1000만의 매트릭스가 형성 
- 한번에 학습 데이터 전체를 진행함 
    - 큰 작업을 처리하기 어려움 
    - GPU 와 같은 병렬처리를 기대하기 힘듬
- 학습을 통해서 개선하기가 어려움

✅ 추론기반의 방법 (Word2Vec) 
추론 : 주변 단어(맥락)이 주어졌을 때 “?”에 무슨 단어(중심단어)가 들어가는지를 추측하는 작업
```

```
✅ 정의 
Word2Vec 은 단어간 유사도를 반영하여 단어를 벡터로 바꿔주는 임베딩 방법론. 
원-핫벡터 형태의 sparse matrix 이 가지는 단점을 해소하고자 저차원의 공간에 벡터로 매핑하는 것이 특징. 
Word2Vec 은 “비슷한 위치에 등장하는 단어들은 비슷한 의미를 가진다“ 라는 가정을 통해서 학습을 진행.
저차원에 학습된 단어의 의미를 분산하여 표현하기에 단어 간 유사도를 계산.
추천시스템에서는 단어를 구매 상품으로 바꿔서 구매한 패턴에 Word2Vec 을 적용해서 비슷한 상품을 찾을 수 있음

✅ 알고리즘 
CBOW : 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법. 
Skip-Gram : 중간에 있는 단어로 주변 단어들을 예측하는 방법.
```

## Word2Vec - CBOW
> you [   ?   ] goodbye and I say hello.
```
1. One Hot Vector 형태의 입력값을 받는다. 
    -  you -> [say] 예측 
    -  goodbye -> [say] 예측 
    -  주변 단어 : 주변에 있는 단어 (you, goodbye) 
    -  중심 단어 : 중간에 있는 단어 (say) 
    -  윈도우 크기 : 주변을 몇 칸까지 볼 지에 대한 크기 (1), 
       만일, 윈도우 크기가 2이면 you, goodbye, and 를 통해서 [say]를 예측
2. One Hot Vector 형태의 입력값을 W_in 과 곱
3. Hidden state 의 값을 W_out 과 곱해서 Score 를 추출한다. 
4. Score 에 Softmax 를 취해서 각 단어가 나올 확률을 계산한다. 
5. 정답과 Cross Entropy Loss 를 계산
6. 5에서 계산한 Loss 를 가지고 Backpropagation 과정을 통해서 Weight 를 업데이트
7. 위의 과정을 다른 문맥에 대해서도 수행
```

## Word2Vec – Skip-gram
> [   ?   ] say [   ?   ] and I say hello.
```
1. One Hot Vector 형태의 입력값을 받는다. 
    -  say -> [you] 예측 
    -  say -> [goodbye] 예측 
    -  주변 단어 : 주변에 있는 단어 (you, goodbye) 
    -  중심 단어 : 중간에 있는 단어 (say) 
    -  윈도우 크기 : 주변을 몇 칸까지 볼 지에 대한 크기 (1), 
       만일, 윈도우 크기가 2이면 say 를 통해 [you], [goodbye], [and]를 예측
2. One Hot Vector 형태의 입력값을 W_in 과 곱
3. Hidden state 의 값을 W_out 과 곱해서 Score 를 추출한다. 
4. Score 에 Softmax 를 취해서 각 단어가 나올 확률을 계산한다. 
5. 정답과 Cross Entropy Loss 를 계산
    - 한 번에 업데이트를 진행
6. 5에서 계산한 Loss 를 가지고 Backpropagation 과정을 통해서 Weight 를 업데이트
    - 두 개의 Answer 에 대해서 오차를 더함
7. 위의 과정을 다른 문맥에 대해서도 수행
```

```
✅ 장점
- 협업필터링은 다른 사용자들의 평점이 필요한 반면에, 자신의 평점만을 가지고 추천시스템을 만들 수 있음 
- item 의 feature 를 통해서 추천을 하기에 추천이 된 이유를 설명하기 용이함 
- 사용자가 평점을 매기지 않은 새로운 item 이 들어올 경우에도 추천이 가능함

✅ 단점
- item 의 feature 을 추출해야 하고 이를 통해서 추천하기때문에 제대로 feature 을 추출하지 못하면 정확도가 낮음. 그렇기에 Domain Knowledge 가 분석시에 필요할 수도 있음 
- 기존의 item 과 유사한 item 위주로만 추천하기에 새로운 장르의 item 을 추천하기 어려움 
- 새로운 사용자에 대해서 충분한 평점이 쌓이기 전까지는 추천하기 힘듬
```

## Word2Vec 알고리즘

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 

import warnings
warnings.filterwarnings(action='ignore')

C:\Users\User\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
movie = pd.read_csv('data/ratings.csv', low_memory=False)
movie.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [3]:
movie = movie.sort_values(by='timestamp', ascending=True).reset_index(drop=True)
movie.head()

,userId,movieId,rating,timestamp
0,383,21,3.0,789652009
1,383,47,5.0,789652009
2,383,1079,3.0,789652009
3,409,21,5.0,828212412
4,409,25,4.0,828212412


In [4]:
# 영화의 Metadata를 불러와서 movieID에 맞는 TITLE을 구해줍니다. 
meta = pd.read_csv('data/movies_metadata.csv', low_memory=False)
meta.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [5]:
meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
meta = meta.rename(columns={'id':'movieId'})
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] = meta['movieId'].astype(str)

movie = pd.merge(movie, meta[['movieId', 'original_title']], how='left', on='movieId')

In [7]:
movie = movie[movie['original_title'].notnull()].reset_index(drop=True)

In [8]:
agg = movie.groupby(['userId'])['original_title'].agg({'unique'})
agg.head()

,unique
userId,
1,"[Jay and Silent Bob Strike Back, Vivement dima..."
2,"[Terminator 3: Rise of the Machines, The Conve..."
3,"[300, The Killing, Shortbus, Finding Neverland..."
4,"[David, The Wedding Planner, Casablanca, Sleep..."
5,"[Gleaming the Cube, Cool Hand Luke, Hidalgo, U..."


In [9]:
movie['original_title'].unique()

array(['The Endless Summer', 'Jarhead', '彼女の想いで', ...,
       'The Lonedale Operator', 'Violeta se fue a los cielos',
       'To Kill a Priest'], dtype=object)

### Word2vec 적용

In [10]:
# int형식은 Word2vec에서 학습이 안되므로 string으로 변경해줍니다. 
sentence = []
for user_sentence in agg['unique'].values:
    sentence.append(list(map(str, user_sentence)))

In [11]:
# Word2vec의 학습을 진행해줍니다. 
from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence, vector_size=20, window = 5, 
                           min_count=1, workers=4, epochs=200, sg=1)

In [12]:
embedding_model.wv.most_similar(positive=['Spider-Man 2'], topn=10)

[('Snow Cake', 0.8682476282119751),
 ('Face/Off', 0.7570157051086426),
 ('Blow', 0.752019464969635),
 ("L'Aile ou la Cuisse", 0.749976396560669),
 ('Star Trek: Nemesis', 0.748651385307312),
 ('Domicile Conjugal', 0.7416670918464661),
 ('Heavenly Creatures', 0.7391387820243835),
 ('Sunrise: A Song of Two Humans', 0.7389362454414368),
 ('Rumor Has It...', 0.7314111590385437),
 ('Conquest of the Planet of the Apes', 0.7279210090637207)]

### Doc2Vec 적용
![image](https://user-images.githubusercontent.com/37262132/121563570-7e01a500-ca55-11eb-89a9-003d6ca55aa1.png)

In [13]:
from gensim.models import doc2vec

In [14]:
meta = pd.read_csv('data/movies_metadata.csv', low_memory=False)
meta = meta[meta['original_title'].notnull()].reset_index(drop=True)
meta = meta[meta['overview'].notnull()].reset_index(drop=True)

In [15]:
from nltk.corpus import stopwords 
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize
import re 
stop_words = set(stopwords.words('english')) 

overview = []
for words in tqdm(meta['overview']):
    word_tokens = word_tokenize(words)
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    sentence = sentence.strip()
    
    sentence_tokens = word_tokenize(sentence)
    result = ''
    for token in sentence_tokens: 
        if token not in stop_words:
            result += ' ' + token 
    result = result.strip().lower()
    overview.append(result)

In [16]:
meta['pre_overview'] = overview

In [17]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm = 0,
    dbow_words = 1,
    window = 10,
    vector_size = 100,
    alpha = 0.025,
    seed = 1234,
    min_count = 5,    
    min_alpha = 0.025,
    workers = 4,
    hs = 1,
    negative = 10
)

In [18]:
from collections import namedtuple

agg = meta[['id', 'original_title', 'pre_overview']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['original_title', 'pre_overview']].values]

In [19]:
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d100,n10,hs,w10,mc5,s0.001,t4)


In [20]:
# 벡터 문서 학습
from time import time

start = time()

for epoch in tqdm(range(5)):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.epochs)
    doc_vectorizer.alpha -= 0.002
    doc_vectorizer.min_alpha = doc_vectorizer.alpha 

end = time()
print(f"During Time: {end-start}")


During Time: 987.6918349266052


In [21]:
doc_vectorizer.docvecs.most_similar('Toy Story', topn=20)

[('Children in the Surf at Coney Island', 0.7353575229644775),
 ('Letzte Worte', 0.725882351398468),
 ('It Stains the Sands Red', 0.6859925985336304),
 ('McLaren', 0.6718501448631287),
 ('По следам бременских музыкантов', 0.6662031412124634),
 ('Skazka o Poteryannom Vremeni', 0.664352536201477),
 ('Kader', 0.6596434712409973),
 ('Wszyscy jesteśmy Chrystusami', 0.6588553786277771),
 ('Przechodzien', 0.6580846309661865),
 ('エクスマキナ', 0.65656977891922),
 ('Dying to Belong', 0.655059278011322),
 ('Begegnung mit Fritz Lang', 0.6500876545906067),
 ('La moutarde me monte au nez', 0.6462430953979492),
 ('El vendedor de humo', 0.6350132822990417),
 ('Особенности национальной политики', 0.6266170144081116),
 ('Live Forever as You Are Now with Alan Resnick', 0.6242215633392334),
 ('Der Sandmann', 0.6195182800292969),
 ('Milk Money', 0.6173915863037109),
 ('Stryapukha', 0.6121023893356323),
 ('Kaos', 0.5985434651374817)]

In [22]:
doc_vectorizer.docvecs.most_similar('Harry Potter and the Deathly Hallows: Part 1', topn=20)

[('The Great Ecstasy of Robert Carmichael', 0.738102376461029),
 ('Handsome Harry', 0.7155303359031677),
 ('Der Räuber', 0.7069940567016602),
 ('Torch Song', 0.682845950126648),
 ('Classe tous risques', 0.6413491368293762),
 ('The Prizefighter and the Lady', 0.6199315190315247),
 ('Tribute', 0.6171056032180786),
 ('Blood River', 0.6087848544120789),
 ('Paternity', 0.6048902273178101),
 ('Three Comrades', 0.6044132113456726),
 ('Just Like Us', 0.6027846336364746),
 ('Steam', 0.602361798286438),
 ('1990: I guerrieri del Bronx', 0.5979241728782654),
 ('Kasaba', 0.597663164138794),
 ('The Librarian: Quest for the Spear', 0.5938440561294556),
 ('Glorious 39', 0.591773509979248),
 ('My Beautiful Dacia', 0.5884438157081604),
 ('Mirrors 2', 0.5798097848892212),
 ('Skupljači perja', 0.573143482208252),
 ('Run of the Arrow', 0.5725336670875549)]